# Probability
## Final project

In the final project you will develop your own classification algorithm based on probabilistic models.

In the `data` folder you can find a collection of texts in English, Italian, Spanish, German, French, Polish and Portuguese languages obtained from random Wikipedia articles, see e.g. `Spanish.txt`. (See corresponding `.source.txt` files and links therein for lists of authors.) 

In [1]:
!ls data

English.sources.txt  German.txt		  Portuguese.sources.txt
English.txt	     Italian.sources.txt  Portuguese.txt
French.sources.txt   Italian.txt	  Spanish.sources.txt
French.txt	     Polish.sources.txt   Spanish.txt
German.sources.txt   Polish.txt


In [2]:
with open("data/Spanish.txt") as f:
    print(f.read(500))

tomas zapata sierra medellin colombia  de mayo de  es productor de cine y de teatrocita requerida
sad eyed lady of the lowlands en espanol senorita de ojos tristes de las tierras bajas es una cancion compuesta por el cantante estadounidense bob dylan fue incluida en el album blonde on blonde editado el  de mayo de 
la revista mojo la coloco en el puesto  de su lista de las  mejores canciones de bob dylan
calyptocephalella canqueli es una especie extinta de anfibio anuro perteneciente al genero c


In [3]:
with open("data/German.txt") as f:
    print(f.read(500))

riedhofe ist der name von ortsteilen in deutschland

in badenwurttemberg
riedhofe langenau ortsteil der stadt langenau im albdonaukreis
riedhofe frickingen ortsteil der gemeinde frickingen im bodenseekreis
riedhofe riegel am kaiserstuhl ortsteil der gemeinde riegel am kaiserstuhl im landkreis emmendingen
riedhofe kongen ortsteil der gemeinde kongen im landkreis esslingen
riedhofe leingarten ortsteil der gemeinde leingarten im landkreis heilbronn
riedhofe bad wurzach ortsteil der stadt bad wurzac


This is our training data. These texts are preprocessed: only standard Latin characters kept, diacritics removed, punctuations removed, all letters converted to lowercase. We will use similar preprocessing for new texts that we have to classify. Here are some useful functions to do it.

### Preprocessing functions

In [4]:
import re
import unicodedata

### FROM: https://stackoverflow.com/a/518232/3025981
def strip_accents(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s)
                   if unicodedata.category(c) != 'Mn')
### END FROM

def clean_text(s):
    return re.sub("[^a-z \n]", "", strip_accents(s))

### Learning: obtaining character frequencies

First of all, we have to find character relative frequencies in texts of each language. We will consider them as probability for character to appear in our multinomial model. Write function `get_freqs(text, relative)` that takes string `text` as input and returns dictionary which keys are all distinct characters occurred in `text` and values are frequencies (relative if `relative` is `True` and absolute otherwise). A similar function was discussed in Python videos previously. Note that Python treat `str` objects as a sequence of characters (e.g. if you try to iterate it).

In [14]:
from collections import Counter


def get_freqs(text, relative=False):
    # your code here
    
    dictionary = Counter(text)
    
    if relative:
        
        number_of_char = sum(dictionary.values())

        for key in dictionary:
            dictionary[key] = dictionary.get(key)/number_of_char
        return dict(dictionary)
    
    return dict(dictionary)  

In [16]:
assert get_freqs('Hello, World!') == {'H': 1, 'e': 1, 'l': 3, 'o': 2, ',': 1,
                                      ' ': 1, 'W': 1, 'r': 1, 'd': 1, '!': 1}

Now use function `get_freqs` to create a dictionary `lang_to_prob` which keys are names of languages (i.e. `'English'`, `'Italian'`, `'Spanish'`, `'German'`, `'French'`, `'Polish'`, `'Portuguese'`) and values are dictionaries of relative frequencies, obtained by processing of corresponding `.txt` files.

In [25]:
# your code here
lang_to_probs = {}
languages = ['English', 'Italian', 'Spanish', 'German', 'French', 'Polish', 'Portuguese']

for language in languages:
    file = "data/" + language + ".txt"
    
    with open(file) as f:
        frequency = get_freqs(clean_text(f.read()), relative=True)
        lang_to_probs[language] = frequency

806999
553868
812937
726934
574219
686563
729104


In [26]:
assert abs(lang_to_probs['Polish']['a'] - 0.08504245058355897) < 0.00001
assert abs(lang_to_probs['English']['x'] - 0.001387857977519179) < 1e-5
assert len(set(lang_to_probs['Portuguese'])) == 28

In [27]:
lang_to_probs

{'English': {'a': 0.07760728328040059,
  'l': 0.03626398545723105,
  'e': 0.09363580376183861,
  'x': 0.001387857977519179,
  'i': 0.06683527488881646,
  ' ': 0.16778707284643476,
  'r': 0.055831543781342974,
  'y': 0.013557637617890481,
  'o': 0.059573803685010765,
  'm': 0.02255145297577816,
  'n': 0.06324295321307709,
  '\n': 0.004925656661284587,
  'v': 0.008451063755965,
  'c': 0.028972774439621363,
  'h': 0.036203266670714586,
  'u': 0.021785652770325615,
  's': 0.05419957149884944,
  'k': 0.006603477823392594,
  'b': 0.01411649828562365,
  'f': 0.018519229887521547,
  'd': 0.030964102805579683,
  't': 0.06663700946345659,
  'g': 0.01564190290198625,
  'w': 0.012623311800882032,
  'p': 0.01732591985863675,
  'j': 0.0023073138876256354,
  'z': 0.0016604729373890178,
  'q': 0.0007881050658055339},
 'Italian': {'n': 0.06224587807925354,
  'e': 0.0949648652747586,
  'c': 0.0347429351397806,
  'h': 0.008155372760296677,
  ' ': 0.15956870590104502,
  'u': 0.027214065445196327,
  't': 0

### Likelihoods
Now let us start implementing the actual classifier. We begin with a multinomial likelihood function. Implement function `multinomial_likelihood(probs, freqs)` that takes two arguments: `probs` are dictionary of probabilities of each character (in some language) and `freqs` is dictionary of absolute frequencies of each character (in some text we want to classify). This function has to return probability to obtain these absolute frequencies from multinomial distribution with given probabilities $P((X_1 = f_1) \cap (X_2 = f_2) \cap \ldots \cap (X_k = f_k))$ provided that $(X_1, \ldots, X_k)$ is a system of multinomially distributed values with probabilities $(p_1, \ldots, p_k)$. (You need the `factorial` function that can be imported from `math`.)


In [46]:
# your code here
import math as m


def multinomial_likelihood(probs, freqs):
    
    n = sum(freqs.values())
    prob = m.factorial(n)
    
    for key in freqs.keys():
        
        p = probs.get(key)
        k = freqs.get(key)
        prob = prob * (p**k)/ m.factorial(k)
    
    return prob

Let's find likelihood of data with frequencies `{'a': 2, 'b': 1, 'c': 2}` and probabilities `{'a': 0.2, 'b': 0.5, 'c': 0.3}`:

In [47]:
multinomial_likelihood(probs={'a': 0.2, 'b': 0.5, 'c': 0.3}, freqs={'a': 2, 'b': 1, 'c': 2})

0.054000000000000006

In [48]:
assert abs(multinomial_likelihood(probs={'a': 0.2, 'b': 0.5, 'c': 0.3},
                    freqs={'a': 2, 'b': 1, 'c': 2}) - 0.054) < 0.000001
assert abs(multinomial_likelihood(probs={'a': 0.2, 'b': 0.1, 'c': 0.3, 'd': 0.4},
                    freqs={'a': 2, 'b': 1, 'c': 2}) - 0.0108) < 0.000001

Note that the coefficient with factorials depends only on `freqs` (i.e. text that we analyse) and does not depend on `probs`. It means that for all possible languages this coefficient will be the same. As we are going to consider fixed text and compare likelihoods for different languages, we see that we do not need this coefficient in most cases. Let us implement the function `multinomial_likelihood_without_coeff` that returns the same probability as `multinomial_likelihood` but without the coefficient.

In [82]:
# your code here
def multinomial_likelihood_without_coeff(probs, freqs):
    
    prob = 1
    
    for key in freqs.keys():
        
        p = probs.get(key)
        k = freqs.get(key)
        prob = prob * (p**k)
    
    return prob

The corresponding probability becomes smaller:

In [83]:
multinomial_likelihood_without_coeff(probs={'a': 0.2, 'b': 0.5, 'c': 0.3},
                                     freqs={'a': 2, 'b': 1, 'c': 2})

0.0018000000000000004

In [51]:
assert multinomial_likelihood_without_coeff(
    probs={'a': 0.3, 'b': 0.4, 'c': 0.3},
    freqs={'a': 2, 'b': 1, 'c': 2}) == 0.00324
assert abs(multinomial_likelihood_without_coeff(
    probs={'a': 0.3, 'b': 0.4, 'c': 0.3},
    freqs={'a': 2, 'b': 1, 'c': 5}) - 8.747999999999e-05) < 1e-10

Actually, likelihoods become extremely small very quickly when we increase absolute frequencies in data. It is not surprising: the probability to get the text that coincides with our actual text from a random experiment we discussed is extremely small.

In [52]:
multinomial_likelihood_without_coeff(probs={'a': 0.2, 'b': 0.5, 'c': 0.3},
                    freqs={'a': 3, 'b': 2, 'c': 2})

0.00018000000000000004

In [53]:
multinomial_likelihood_without_coeff(probs={'a': 0.2, 'b': 0.5, 'c': 0.3},
                    freqs={'a': 3, 'b': 20, 'c': 2})

6.866455078125001e-10

Due to limited precision of computer arithmetic, for frequencies large enough we will get exactly zero likelihood.

In [54]:
multinomial_likelihood_without_coeff(probs={'a': 0.2, 'b': 0.5, 'c': 0.3},
                                     freqs={'a': 543, 'b': 512, 'c': 2})

0.0

Thus we usually cannot use likelihoods like this directly. Common way to deal with such a tiny numbers is to use _logarithms_ instead of likelihood themself. Indeed, logarithm is a monotonically increasing function. If we compare logarithms, it is equivalent to compare their arguments.

### Log likelihood
Implement function `log_likelihood_without_coeff(probs, freqs)` that calculates logarithm of likelihood (without factorial coefficient). Note that you cannot simply put `multinomial_likelihood_without_coeff` into `log`: if the likelihood would be extremely small (equal to zero from the computer's point of view), `log` of it will be negative infinity. Thus we have to algebraically transform the expression for log likelihood first. Let us use properties of logarithm to do it: $\log (ab) = \log a + \log b$, $\log (a^b)=b\log a$.

In [59]:
# your code here
import math as m


def log_likelihood_without_coeff(probs, freqs):
    
    prob_log = 0
    
    for key in freqs.keys():
        
        p = probs.get(key)
        k = freqs.get(key)
        prob_log += k * m.log(p)
    
    return prob_log

Likelihoods are probabilities, so they are less than 1 and their logarithms are negative. The larger absolute value of log-likelihood, the less is likelihood. 

In [60]:
log_likelihood_without_coeff(probs={'a': 0.2, 'b': 0.5, 'c': 0.3},
                             freqs={'a': 2, 'b': 1, 'c': 2})

-6.319968614080018

Now we can deal with the inputs that lead to exact zero value previously.

In [61]:
log_likelihood_without_coeff(probs={'a': 0.2, 'b': 0.5, 'c': 0.3},
                             freqs={'a': 543, 'b': 512, 'c': 2})

-1231.2240885070605

In [62]:
assert abs(log_likelihood_without_coeff(probs={'a': 0.2, 'b': 0.5, 'c': 0.3},
                              freqs={'a': 2, 'b': 1, 'c': 2}) + 6.319968614080018) < 0.00001
assert abs(log_likelihood_without_coeff(probs={'a': 0.2, 'b': 0.5, 'c': 0.3},
                             freqs={'a': 543, 'b': 512, 'c': 2}) + 1231.2240885070605) < 0.0001
assert abs(log_likelihood_without_coeff(probs={'a': 0.2, 'b': 0.5, 'c': 0.3},
                             freqs={'a': 543, 'b': 512}) + 1228.8161428984085) < 0.0001

### Theoretical question: cross-entropy and Jensen's inequality

The function that you obtained is also known as *cross entropy* and is extremely popular in machine learning, namely, in classification problems. It allows you to measure how good probability distribution $(p_1, \ldots, p_k)$ fits to actual absolute frequencies obtained from the data $(f_1, \ldots, f_k)$.

Assume that frequencies $(f_1, \ldots, f_k)$ are fixed. What is the best distribution $(p_1, \ldots, p_k)$ from likelihood's perspective?

Intuitively, it seems that we have to put relative frequencies
$$r_i = \frac{f_i}{\sum_{j=1}^k f_j}$$ 
as $p_i$ to get best fit. In fact, it is true. To prove it, let us use Jensen's inequality for logarithms. It is stated as follows:

For any values $\alpha_1, \ldots, \alpha_k$, such that $\sum_{j=1}^k \alpha_j = 1$ and $\alpha_j \ge 0$ for all $j=1, \ldots, k$, and any positive values $x_1, \ldots, x_k$, the following inequality holds:

$$\log \sum_{j=1}^k \alpha_j x_j \ge \sum_{j=1}^k \alpha_j\log(x_j).$$

Use this inequality to prove that
$$\sum_{j=1}^k r_j \log p_j - \sum_{j=1}^k r_j \log r_j \le 0,$$
then prove that to obtain maximum log-likelihood (and therefore maximum likelihood) for fixed $(f_1, \ldots, f_k)$ we have to put $p_i=r_i$, $i=1,\ldots, k$.

**Hint:** use properties of logarithm to transform the left-hand part of the last inequality to the right-hand part of the previous inequality.

(Submit your proof as a staff graded assignment.)

### Maximum likelihood classifier

Now we will use likelihood to choose the best language for some text we want to classify. Write function `mle_best(text, lang_to_probs)` that takes some `text` and dictionary `lang_to_probs` that we created previously and returns the name of the language such that the likelihood of our data for this language is maximal. Note that you have to preprocess  `text` using the function `clean_text` before finding frequencies.

In [72]:
# your code here
import math as m


def mle_best(text, lang_to_probs): 
    dictionary = get_freqs(clean_text(text))
    
    max_prob = -m.inf
    
    for key in lang_to_probs.keys():
        
        prob = log_likelihood_without_coeff(lang_to_probs[key], dictionary)
        
        if prob > max_prob:
            language = key
            max_prob = prob
                    
    return language 


Let's test it!

In [73]:
# Source: https://en.wikipedia.org/wiki/1134_Kepler
text = """1134 Kepler, provisional designation 1929 SA, is a stony asteroid 
and eccentric Mars-crosser from the asteroid belt, approximately 
4 kilometers in diameter"""
mle_best(text, lang_to_probs)

'English'

In [74]:
# Source: https://pl.wikipedia.org/wiki/(1134)_Kepler
text = """"(1134) Kepler – planetoida z grupy przecinających 
orbitę Marsa okrążająca Słońce w ciągu 4 lat i 145 dni 
w średniej odległości 2,68 au.
"""
mle_best(text, lang_to_probs)

'Polish'

In [75]:
# Source: https://it.wikipedia.org/wiki/1134_Kepler
text = """1134 Kepler è un asteroide areosecante. Scoperto nel 1929, 
presenta un'orbita caratterizzata da un semiasse maggiore pari a 2,6829098 
UA e da un'eccentricità di 0,4651458, inclinata di 15,17381° rispetto
"""
mle_best(text, lang_to_probs)

'Italian'

In [76]:
lines = ['Aaa', 'Aa', 'Kepler è un asteroide areosecante. Scoperto nel',
        "presenta un'orbita caratterizzata da un semiasse maggiore pari a 2,6829098 "
         "UA e da un'eccentricità di 0,4651458, inclinata",
         "Kepler – planetoida z grupy przecinających orbitę Marsa okrążająca Słońce w ciągu 4 lat i ",
         "Kepler, provisional designation 1929 SA, is a stony asteroid "
         "and eccentric Mars-crosser from the"]
assert mle_best(lines[0], lang_to_probs) == 'Portuguese'
assert mle_best(lines[1], lang_to_probs) == 'Portuguese'
assert mle_best(lines[2], lang_to_probs) == 'French'
assert mle_best(lines[3], lang_to_probs) == 'Italian'
assert mle_best(lines[4], lang_to_probs) == 'Polish'
assert mle_best(lines[5], lang_to_probs) == 'English'
assert mle_best("Aaaa", lang_to_probs) == "Portuguese"

### Look, it works!
Fantastic! Just a couple of equations, and we created fully automatic language detection algorithm!

Now let us make it even better.

### Let's go Bayes
Assume that we selected random article from all Wikipedia articles in the languages that we consider. There are different numbers of articles in different languages, so it is more likely to obtain an article e.g. in English than in Polish (at least, at this moment). It means that we need stronger evidence for Polish to accept it compared with English. To take into account this information, we will use Bayes' rule as discussed in the videos.

Recall that in the Bayesian approach we consider _prior_ probabilities of languages, then use Bayes' rule to find their posterior probabilities and select the language with the largest posterior probability. We begin with finding prior probabilities. Create a dictionary `lang_to_prior` whose keys are language names and values are prior probabilities. Assume that priors are proportional to the number of articles in each language (in thousands). Let us use these numbers (in thousands): English: 6090, Italian: 1611, Spanish: 1602, German: 2439, French: 2222, Polish: 1412, Portuguese: 1034. Remember that all prior probabilities should sum up to 1!

In [77]:
# your code here
lang_to_prior = {'English': 6090,
                 'Italian': 1611,
                 'Spanish': 1602,
                 'German': 2439,
                 'French': 2222,
                 'Polish': 1412,
                 'Portuguese': 1034}

total_number_of_articles = sum(lang_to_prior.values())

for language in lang_to_prior:
    lang_to_prior[language] = lang_to_prior[language] / total_number_of_articles


In [78]:
assert abs(lang_to_prior['French'] - 0.13540524070688603) < 0.000001

Now implement function `bayesian_best(text, lang_to_probs, lang_to_prior)` that takes some text `text`, dictionary `lang_to_probs` created before and `lang_to_prior` with prior probabilities. This function has to return the language name with the largest posterior probability. Note that as we only compare posterior probabilities, we can ignore the denominator in Bayes' rule: it is the same for all languages. Note also that we need to work with logarithms of posterior instead of posteriors themself to avoid extremely small numbers. Use properties of logarithm to do it efficiently.

In [87]:
# your code here
import math as m


def bayesian_best(text, lang_to_probs, lang_to_prior):
    frequency = get_freqs(clean_text(text))   
    
    languages = ['English', 'Italian', 'Spanish', 'German', 'French', 'Polish', 'Portuguese']
    
    max_prob = -m.inf
    
    for language in languages:
        # count the probabilities
        likelihood = multinomial_likelihood_without_coeff(lang_to_probs[language], frequency)
        lang_prob = lang_to_prior[language]
        
        prob = m.log(likelihood) + m.log(lang_prob)
        
        if prob > max_prob:
            likely_language = language
            max_prob = prob
            
    return likely_language

For example, MLE algorithm believes that word `"The"` belongs go German language.

In [88]:
mle_best("The", lang_to_probs)

'German'

However, if we take into account that English is more popular in Wikipedia, the results changes:

In [89]:
bayesian_best("The", lang_to_probs, lang_to_prior)

'English'

In [90]:
lines = ['Aaa', 'Aa', 'Kepler è un asteroide areosecante. Scoperto nel',
        "presenta un'orbita caratterizzata da un semiasse maggiore pari a 2,6829098 "
         "UA e da un'eccentricità di 0,4651458, inclinata",
         "Kepler – planetoida z grupy przecinających orbitę Marsa okrążająca Słońce w ciągu 4 lat i ",
         "Kepler, provisional designation 1929 SA, is a stony asteroid "
         "and eccentric Mars-crosser from the"]
answers = ['English', 'English', 'French', 'Italian', 'Polish', 'English']
for line, answer in zip(lines, answers):
    assert bayesian_best(line, lang_to_probs, lang_to_prior) == answer

### Measuring uncertainty
Finally, let us get posterior probability of how certain our Bayesian algorithm is in its classification. Implement function `bayesian_posterior(text, lang_to_probs, lang_to_prior, test_lang)` that takes some `text`, `lang_to_probs`, `lang_to_prior` and a particular language `test_lang` we are interested in and returns posterior probability for this language provided this text. Use formula for the posterior from the videos.

Note that in this case you have to work with actual probabilities (likelihoods), not their logarithms.

In [92]:
# your code here
def bayesian_posterior(text, lang_to_probs, lang_to_prior, test_lang):
    freqs = get_freqs(clean_text(text))   
    
    languages = ['English', 'Italian', 'Spanish', 'German', 'French', 'Polish', 'Portuguese']
    
    prob_dict = {}
    
    for language in languages:

        likelihood = multinomial_likelihood_without_coeff(lang_to_probs[language], freqs)
        lang_prob = lang_to_prior[language]

        prob = likelihood * lang_prob
        prob_dict[language] = prob
        
    return prob_dict[test_lang] / sum(prob_dict.values())

In [93]:
bayesian_posterior("The", lang_to_probs, lang_to_prior, "German")

0.2686381464045339

In [94]:
bayesian_posterior("The", lang_to_probs, lang_to_prior, "English")

0.534626604689872

In [95]:
bayesian_posterior("Das", lang_to_probs, lang_to_prior, "English")

0.36596793151737517

In [96]:
bayesian_posterior("Das", lang_to_probs, lang_to_prior, "German")

0.12095519926831602

In [97]:
assert abs(bayesian_posterior("The", lang_to_probs, lang_to_prior, "German") - 0.26863814640) < 0.00001
assert abs(bayesian_posterior("The", lang_to_probs, lang_to_prior, "English") - 0.534626604) < 0.00001
assert abs(bayesian_posterior("Das", lang_to_probs, lang_to_prior, "English") - 0.365967931517) < 0.00001

We see that our algorithm believes that `"Das"` belongs to English. This is due to the small amount of data (only three letters!) and high prior for English. However, it is not very certain: the posterior is only 0.37. On the other hand, `"The"` belongs to `"English"` with much larger posterior probability.

## Conclusions
Today we used our knowledge of probability to construct our own classifier algorithm. Actually, it is a version of a well-known naive Bayesian classifier. Of course, this classifier is far from perfect: for example, it completely ignores words and deals only with characters and their frequencies. Nevertheless, it works rather well despite being so simple. Also it shows several important concepts: likelihood, maximum likelihood estimation, Bayesian estimations and so on.

Now you are ready for more sophisticated topics.

### P.S. Efficiency considerations
During this project we used dictionaries to represent frequency tables and pure Python constructions like loops to process them. However, it is much more efficient to use `numpy` arrays and vectorized functions. You can try to redo this project with `numpy` if you know how to use it. If you don't know yet, that's not a problem: you will learn it soon.